In [103]:
import re
from typing import Any

from langchain_community.chat_models import ChatTongyi

# ruff: noqa: F401
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.tools import tool
from langchain_ollama.chat_models import ChatOllama

from citra.mcp.tool import get_connection

In [104]:
conn = get_connection()


@tool
def sql_outage(sql: str) -> str:
    """生成关于停电信息的sql查询语句。表结构：TABLE( \
        t_event_alarm_inter (equipName设备名称,lineName线路名称,faultType停电类型,\
        occurTime停电开始时间,endTime停电结束时间,
        gdsName 供电所单位名称，unitName供电公司名称)。详情只输出以上字段"""
    sql = sql.replace('`', '').replace('\\n', ' ')
    search = re.search(r'((select|SELECT)[^"]+)', sql)
    if search:
        return search.group()
    return ''


@tool
def sql_order_fault(sql: str) -> str:
    """生成关于工单故障的sql查询语句。表结构为 TABLE t_fault_order_inter (
    work_order_id 工单号,  received_time 受理时间,  address  地址,
    issue_description  受理内容,  phone_number 联系电话,  contact_person  联系人,
    analysis_result  分析结果,  order_type  工单类型,  unit_name  单位,
    power_supply_station 供电所）。详情只输出以上字段"""
    sql = sql.replace('`', '').replace('\\n', ' ')
    search = re.search(r'((select|SELECT)[^"]+)', sql)
    if search:
        return search.group()
    return ''

In [ ]:
def to_markdown(cols: list, rows: tuple[tuple[Any, ...], ...]):
    if len(cols) != len(rows[0]):
        raise ValueError('列数和行数不匹配')
    yield '| ' + ' | '.join(cols) + ' |\n'
    yield '| ' + ' | '.join(['---'] * len(cols)) + ' |\n'
    for row in rows:
        yield '| ' + ' | '.join(str(cell) for cell in row) + ' |\n'


def str_to_gen(s: str, *, msg_type: str = 'msg', chunk_size: int = 10):
    """将字符串转换为生成器"""
    for i in range(0, len(s), chunk_size):
        yield s[i : i + chunk_size]


def execute_sql(sql: str):
    """执行sql语句，返回结果"""

    if sql == '':
        return '生成sql失败'
    with conn.cursor() as cursor:
        try:
            cursor.execute(sql)
            results = cursor.fetchall()
            if not results:
                yield '没有查询数据'
                return
            cols = [desc[0] for desc in cursor.description]
            yield from to_markdown(cols, results)
        except Exception as e:
            return f'执行sql失败: {e}'

In [112]:
# chatllm = ChatTongyi(model='qwen-max', temperature=0.7)
chatllm = ChatOllama(model='qwen2.5:3b', temperature=0.7)
tools = [sql_outage, sql_order_fault]

chat_with_tools = chatllm.bind_tools(tools)


def call_tools(msg: AIMessage) -> list[dict]:
    """Simple sequential tool calling helper."""
    tool_map = {tool.name: tool for tool in tools}
    tool_calls = msg.tool_calls.copy()
    for tool_call in tool_calls:
        tool_call['output'] = tool_map[tool_call['name']].invoke(tool_call['args'])
    return tool_calls


def ask_question(question: str):
    """问数据库问题"""
    try:
        ai_msg = chat_with_tools.invoke([HumanMessage(content=question)])
        if not ai_msg.tool_calls:
            print('没有调用工具')
            yield from str_to_gen(ai_msg.content)
        else:
            print('调用工具')
            query = call_tools(ai_msg)[0]['output']
            yield from execute_sql(query)
    except Exception as e:
        print(f'遇到错误: {e}')

In [ ]:
for i in ask_question('请给出江山供电公司的故障信息'):
    print(i, end='')

调用工具
| work_order_id | received_time | address | issue_description | phone_number | contact_person | analysis_result | order_type | unit_name | power_supply_station |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 2505130435418982 | 2025-05-13 04:35:41 | 浙江省衢州市江山市石门镇东坑村88号 | 受理时间：2025-05-13 04:35:39，用户编号：，联系人：先生,联系电话：13867025042，故障地址：浙江省衢州市江山市石门镇东坑村88号，受理内容：一户无电。 | 13867025042 | 先生 | 【工单】近90天，该用户涉及95598工单0张，数供工单1张； | 外联故障工单 | 江山供电公司 | 清漾供电所 |
| 2505130442125808 | 2025-05-13 04:42:12 | 浙江省衢州市江山市峡口镇大峦口村东坑村88号 | 受理时间：2025-05-13 04:42:08，用户编号：，联系人：先生,联系电话：13867025042，故障地址：浙江省衢州市江山市峡口镇大峦口村东坑村88号，受理内容：一户无电。 | 13867025042 | 先生 | 【工单】近90天，该用户涉及95598工单0张，数供工单2张； | 外联故障工单 | 江山供电公司 | 峡口供电所 |
| 2505130728071999 | 2025-05-13 07:28:07 | 浙江省衢州市江山市虎山街道江东社区江东大道江东五区小区109幢1单元302室 | 受理时间：2025-05-13 07:28:05，用户编号：3307710047787，联系人：江山市房地产管理处,联系电话：13587018406，故障地址：浙江省衢州市江山市虎山街道江东社区江东大道江东五区小区109幢1单元302室，受理内容：一户无电。 | 13587018406 | 江山市房地产管理处 | 【工单】近90天，该用户涉及95598工单0张，数供工单1张； | 外联故障工单 | 江山供电公司 |